## imports

In [2]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import pickle
import warnings
warnings.filterwarnings("ignore")
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
from matplotlib import gridspec
plt.ion()
import pandas as pd
from coastsat import SDS_download, SDS_preprocess, SDS_shoreline, SDS_tools, SDS_transects,SDS_islands,SDS_universal_transects,SDS_preprocess_med,SDS_earthcache_api
from osgeo import gdal
import json
print(os.getcwd())

d:\Github\CoastSat


## reformat downloads

In [3]:
SDS_earthcache_api.reformat_downloads('Ukulhas_', 'ukulhas_ps')

<module 'ntpath' from 'c:\\Users\\walte\\.conda\\envs\\coastnurbs\\lib\\ntpath.py'>
<module 'ntpath' from 'c:\\Users\\walte\\.conda\\envs\\coastnurbs\\lib\\ntpath.py'>
<module 'ntpath' from 'c:\\Users\\walte\\.conda\\envs\\coastnurbs\\lib\\ntpath.py'>
<module 'ntpath' from 'c:\\Users\\walte\\.conda\\envs\\coastnurbs\\lib\\ntpath.py'>
<module 'ntpath' from 'c:\\Users\\walte\\.conda\\envs\\coastnurbs\\lib\\ntpath.py'>
<module 'ntpath' from 'c:\\Users\\walte\\.conda\\envs\\coastnurbs\\lib\\ntpath.py'>
<module 'ntpath' from 'c:\\Users\\walte\\.conda\\envs\\coastnurbs\\lib\\ntpath.py'>
<module 'ntpath' from 'c:\\Users\\walte\\.conda\\envs\\coastnurbs\\lib\\ntpath.py'>
<module 'ntpath' from 'c:\\Users\\walte\\.conda\\envs\\coastnurbs\\lib\\ntpath.py'>
<module 'ntpath' from 'c:\\Users\\walte\\.conda\\envs\\coastnurbs\\lib\\ntpath.py'>
<module 'ntpath' from 'c:\\Users\\walte\\.conda\\envs\\coastnurbs\\lib\\ntpath.py'>
<module 'ntpath' from 'c:\\Users\\walte\\.conda\\envs\\coastnurbs\\lib\\ntpa

## testing band selection

In [88]:
fn_ms = 'data\\ukulhas_ps\\S2\\ms\\2023-10-07-05-02-03-PLSD_ukulhas_ps_ms.tif'
fn_meta = fn_ms.replace('_ms.tif','.txt').replace('ms','meta') 
bandlist= SDS_preprocess_med.get_bandlist(fn_meta)
bandlist

['coastal-blue',
 'blue',
 'green053',
 'green',
 'yellow',
 'red',
 'vegetation-red-edge-1',
 'near-infrared',
 'flags',
 'tmp_flags',
 'mosaic_flags\n']

In [89]:
band_order = ['blue', 'green','red', 'near-infrared','short-wave-infrared-1', 'short-wave-infrared-2','red']
im_ms, georef, cloud_mask, im_extra, im_QA, im_nodata = SDS_preprocess_med.preprocess(fn_ms,'S2',band_order)
im_ms.shape

Missing bands: ['short-wave-infrared-1', 'short-wave-infrared-2']
unused bands: ['coastal-blue', 'green053', 'yellow', 'vegetation-red-edge-1', 'flags', 'tmp_flags', 'mosaic_flags\n']


(750, 741, 5)

## read in settings and start coastsat process

In [4]:
#read pkl file
with open('data\\ukulhas_ps\\ukulhas_ps_metadata.pkl', 'rb') as f:
    data = pickle.load(f)
    print(data.keys())
    print(data['S2'])

dict_keys(['S2'])
['2023-01-01-05-01-06-PLSD_ukulhas_ps_ms.tif', '2023-01-03-04-03-05-PLSD_ukulhas_ps_ms.tif', '2023-01-08-05-01-01-PLSD_ukulhas_ps_ms.tif', '2023-01-14-04-03-03-PLSD_ukulhas_ps_ms.tif', '2023-01-17-05-00-09-PLSD_ukulhas_ps_ms.tif', '2023-01-22-05-01-02-PLSD_ukulhas_ps_ms.tif', '2023-01-31-04-03-06-PLSD_ukulhas_ps_ms.tif', '2023-02-16-05-01-02-PLSD_ukulhas_ps_ms.tif', '2023-03-28-04-03-03-PLSD_ukulhas_ps_ms.tif', '2023-04-18-04-03-03-PLSD_ukulhas_ps_ms.tif', '2023-05-05-04-04-00-PLSD_ukulhas_ps_ms.tif', '2023-05-08-05-01-04-PLSD_ukulhas_ps_ms.tif', '2023-05-17-05-01-04-PLSD_ukulhas_ps_ms.tif', '2023-06-15-05-01-07-PLSD_ukulhas_ps_ms.tif', '2023-06-17-04-03-07-PLSD_ukulhas_ps_ms.tif', '2023-07-09-04-03-04-PLSD_ukulhas_ps_ms.tif', '2023-07-19-05-01-06-PLSD_ukulhas_ps_ms.tif', '2023-07-22-05-02-00-PLSD_ukulhas_ps_ms.tif', '2023-07-28-04-03-09-PLSD_ukulhas_ps_ms.tif', '2023-08-01-04-04-03-PLSD_ukulhas_ps_ms.tif', '2023-08-07-04-03-07-PLSD_ukulhas_ps_ms.tif', '2023-08-12-04-

In [5]:
# region of interest
polygon = SDS_tools.polygon_from_kml(os.path.join(os.getcwd(), 'example','ukulhas.kml'))
#polygon = aoi
# date range
dates = ['2022-01-01', '2023-12-30']
# satellite missions
sat_list = ['S2']
# choose Landsat collection 'C01' or 'C02'
collection = 'C02'
# name of the site
sitename = 'ukulhas_ps'#'Madhirivaadhoo_ps'
# directory where the data will be stored
filepath = os.path.join(os.getcwd(), 'data')
# put all the inputs into a dictionnary
inputs = {'polygon': polygon, 'dates': dates, 'sat_list': sat_list, 'sitename': sitename, 'filepath':filepath,
         'landsat_collection': collection}

In [6]:
metadata = SDS_download.get_metadata(inputs) 

In [18]:
# settings for the sand contour mapping
settings = { 
    # general parameters:
    'cloud_thresh': 0.5,        # threshold on maximum cloud cover
    'dist_clouds': 300,         # ditance around clouds where shoreline can't be mapped
    'output_epsg': 4326,#3857,        # epsg code of spatial reference system desired for the output
    # quality control:
    'check_detection': False,    # if True, shows each shoreline detection to the user for validation
    'adjust_detection': False,  # if True, allows user to adjust the postion of each shoreline by changing the threhold        
    'check_detection_sand_poly': False, # if True, uses sand polygon for detection and shows user for validation 
    'save_figure': True,               # if True, saves a figure showing the mapped shoreline for each image
    # add the inputs defined previously
    'inputs': inputs,
    # [ONLY FOR ADVANCED USERS] shoreline detection parameters:
    'min_beach_area': 10,        # minimum area (in metres^2) for an object to be labelled as a beach
    'buffer_size': 20,         # radius (in metres) of the buffer around sandy pixels considered in the shoreline detection
    'min_length_sl': 50,       # minimum length (in metres) of shoreline perimeter to be valid
    'cloud_mask_issue': False,  # switch this parameter to True if sand pixels are masked (in black) on many images
    'sand_color': 'default',    # 'default', 'dark' (for grey/black sand beaches) or 'bright' (for white sand beaches)
    'pan_off': False,           # True to switch pansharpening off for Landsat 7/8/9 imagery
    's2cloudless_prob': 60,     # probability threshold to identify cloudy pixels in the s2cloudless mask
}

## save jpegs

In [10]:
SDS_preprocess_med.save_jpg(metadata, settings)

Saving images as jpg:
S2: 49 images
2%Missing bands: []
unused bands: ['coastal-blue', 'vegetation-red-edge-1', 'vegetation-red-edge-2', 'vegetation-red-edge-3', 'narrow-near-infrared', 'water-vapor', 'average-scene-water-vapor', 'aerosol-optical-thickness', 'flags', 'mosaic_flags\n']
4%Missing bands: []
unused bands: ['coastal-blue', 'vegetation-red-edge-1', 'vegetation-red-edge-2', 'vegetation-red-edge-3', 'narrow-near-infrared', 'water-vapor', 'average-scene-water-vapor', 'aerosol-optical-thickness', 'flags', 'mosaic_flags\n']
6%Missing bands: []
unused bands: ['coastal-blue', 'vegetation-red-edge-1', 'vegetation-red-edge-2', 'vegetation-red-edge-3', 'narrow-near-infrared', 'water-vapor', 'average-scene-water-vapor', 'aerosol-optical-thickness', 'flags', 'mosaic_flags\n']
8%Missing bands: ['near-infrared', 'short-wave-infrared-1', 'short-wave-infrared-2']
unused bands: ['coastal-blue', 'green053', 'yellow', 'vegetation-red-edge-1', 'flags', 'tmp_flags', 'mosaic_flags\n']
10%Missing 

## extract shorelines

In [19]:
%matplotlib qt
settings['reference_shoreline'] = SDS_preprocess_med.get_reference_sl(metadata, settings)
settings['max_dist_ref'] = 50 # max distance (in meters) allowed from the reference shoreline

Reference shoreline already exists and was loaded
Reference shoreline coordinates are in epsg:4326


In [20]:
#if periodic, set the last point equal to the first point
settings['periodic'] = True
# fit a nurbs curve through the shoreline
settings['reference_nurbs'] = SDS_universal_transects.fit_nurbs(settings['reference_shoreline'],degree=3,size=24,periodic=settings['periodic'])

In [21]:
%matplotlib inline
output = SDS_preprocess_med.extract_shorelines(metadata, settings)

Mapping shorelines:
S2:   2%Missing bands: []
unused bands: ['coastal-blue', 'vegetation-red-edge-1', 'vegetation-red-edge-2', 'vegetation-red-edge-3', 'narrow-near-infrared', 'water-vapor', 'average-scene-water-vapor', 'aerosol-optical-thickness', 'flags', 'mosaic_flags\n']
S2:   4%Missing bands: []
unused bands: ['coastal-blue', 'vegetation-red-edge-1', 'vegetation-red-edge-2', 'vegetation-red-edge-3', 'narrow-near-infrared', 'water-vapor', 'average-scene-water-vapor', 'aerosol-optical-thickness', 'flags', 'mosaic_flags\n']
S2:   6%Missing bands: []
unused bands: ['coastal-blue', 'vegetation-red-edge-1', 'vegetation-red-edge-2', 'vegetation-red-edge-3', 'narrow-near-infrared', 'water-vapor', 'average-scene-water-vapor', 'aerosol-optical-thickness', 'flags', 'mosaic_flags\n']
S2:   8%Missing bands: ['near-infrared', 'short-wave-infrared-1', 'short-wave-infrared-2']
unused bands: ['coastal-blue', 'green053', 'yellow', 'vegetation-red-edge-1', 'flags', 'tmp_flags', 'mosaic_flags\n']
S2:

In [22]:
output = SDS_tools.remove_duplicates(output) # removes duplicates (images taken on the same date by the same satellite)
output = SDS_tools.remove_inaccurate_georef(output, 10) # remove inaccurate georeferencing (set threshold to 10 m)

0 duplicates
0 bad georef


In [23]:
filepath = os.path.join(inputs['filepath'], sitename)
with open(os.path.join(filepath, sitename + '_output' + '.pkl'), 'rb') as f:
    output = pickle.load(f)

In [24]:
print(len(output['dates']))
output = SDS_universal_transects.remove_gaps(output,100)
print(len(output['dates']))

49
0 shorelines with gaps removed
49


In [27]:
from datetime import datetime

csvpath = os.path.join(inputs['filepath'], sitename)
csvpath = csvpath +"/"+ sitename + '_output.csv'

#export boundaries as csv
shorelines = output['shorelines']
dates = output['dates']
for i in range(len(shorelines)):
    distances = [datetime.strftime(dates[i],'%Y-%m-%d')] 
    for j in range(len(shorelines[i])):
        distances.append(str(shorelines[i][j])) 
    SDS_universal_transects.add_line_to_csv(distances,csvpath)

SDS_universal_transects.infill_missing_days(csvpath)